In [1]:
from openai import OpenAI

In [4]:
from dotenv import load_dotenv

In [5]:
load_dotenv()

True

In [7]:
import os

In [8]:
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

In [13]:
response = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [
        {
            "role": "system", "content": f"""
            You are an expert in categorising IT support tickets. Given the support ticket, you need categorise each ticket into one of the 
            categories listed below:
            1. Hardware
            2. Software
            3. Others
    
            Response in one of the above words only. 
            """},
        {
            "role": "user", "content": f"My monitor won't turn on"
        }
    ], temperature = 0
)

print(response.choices[0].message.content)

Hardware


In [16]:
eval_test = client.evals.create(
    name = "IT Ticket Categorisation",
    data_source_config = {
        "type": "custom",
        "item_schema": {
            "type": "object",
            "properties": {
                "ticket_text": {"type": "string"},
                "correct_label": {"type": "string"}
            },
            "required": ["ticket_text", "correct_label"]
        },
        "include_sample_schema": True
    },

    testing_criteria = [
        {
            "type": "string_check",
            "name": "Match to human label",
            "input": "{{sample.output_text}}",
            "operation": "eq",
            "reference": "{{item.correct_label}}"
        }
    ]
)

print(eval_test)

EvalCreateResponse(id='eval_6940b036ee948191a812f11f980d4935', created_at=1765847094, data_source_config=EvalCustomDataSourceConfig(schema_={'type': 'object', 'properties': {'item': {'type': 'object', 'properties': {'ticket_text': {'type': 'string'}, 'correct_label': {'type': 'string'}}, 'required': ['ticket_text', 'correct_label']}, 'sample': {'type': 'object', 'properties': {'model': {'type': 'string'}, 'choices': {'type': 'array', 'items': {'type': 'object', 'properties': {'message': {'type': 'object', 'properties': {'role': {'type': 'string', 'enum': ['assistant']}, 'content': {'type': ['string', 'array', 'null']}, 'refusal': {'type': ['boolean', 'null']}, 'tool_calls': {'type': ['array', 'null'], 'items': {'type': 'object', 'properties': {'type': {'type': 'string', 'enum': ['function']}, 'function': {'type': 'object', 'properties': {'name': {'type': 'string'}, 'arguments': {'type': 'string'}}, 'required': ['name', 'arguments']}, 'id': {'type': 'string'}}, 'required': ['type', 'fun

In [15]:
file_upload = client.files.create(
    file = open("/Users/Rakshit.Lodha/evals/tickets.jsonl","rb"),
    purpose = "evals"
)

print(file_upload)

FileObject(id='file-BW89c77fZFoMrq8dG4Xb3k', bytes=268, created_at=1765846820, filename='tickets.jsonl', object='file', purpose='evals', status='processed', expires_at=None, status_details=None)


In [17]:
eval_run = client.evals.runs.create(
    "eval_6940b036ee948191a812f11f980d4935",
    name = "Categorisation",
    data_source = {
        "type": "responses",
        "model": "gpt-3.5-turbo",
        "input_messages": {
            "type": "template",
            "template": [
                {"role": "developer", "content": "You are an expert in categorizing IT support tickets. Given the support ticket below, categorize the request into one of 'Hardware', 'Software', or 'Other'. Respond with only one of those words."},
                {"role": "user", "content": "{{ item.ticket_text }}"},
            ]
        },
        "source": {"type": "file_id", "id": "file-BW89c77fZFoMrq8dG4Xb3k"}
    }
)

print(eval_run)

RunCreateResponse(id='evalrun_6940b117c0f481919adf813d0d40f93f', created_at=1765847319, data_source=DataSourceResponses(source=DataSourceResponsesSourceFileID(id='file-BW89c77fZFoMrq8dG4Xb3k', type='file_id'), type='responses', input_messages=DataSourceResponsesInputMessagesTemplate(template=[DataSourceResponsesInputMessagesTemplateTemplateEvalItem(content="You are an expert in categorizing IT support tickets. Given the support ticket below, categorize the request into one of 'Hardware', 'Software', or 'Other'. Respond with only one of those words.", role='developer', type='message'), DataSourceResponsesInputMessagesTemplateTemplateEvalItem(content='{{ item.ticket_text }}', role='user', type='message')], type='template'), model='gpt-3.5-turbo', sampling_params=None), error=None, eval_id='eval_6940b036ee948191a812f11f980d4935', metadata={}, model='gpt-3.5-turbo', name='Categorisation', object='eval.run', per_model_usage=None, per_testing_criteria_results=None, report_url='https://platfo